# define all locations, HRRR Variables, and time range, then fetch data

In [ ]:
from pathlib import Path
from datetime import datetime

from pyhrrrzarr.schema import csv_to_locations
from pyhrrrzarr.hrrr.schema import HRRRVariable, VariableName, Level 
from pyhrrrzarr.hrrrzarr.requests import create_requests


csv_path = Path("../data/WiscoNet pre-2024 stations.csv").resolve()
locations = csv_to_locations(csv_path)
vars = [
    HRRRVariable(name=VariableName.TMP, level=Level._2M_ABOVE_GROUND),
    HRRRVariable(name=VariableName.DPT, level=Level._2M_ABOVE_GROUND),
    HRRRVariable(name=VariableName.APCP_1hr_acc_fcst, level=Level._SURFACE, type_model="fcst"),
    HRRRVariable(name=VariableName.RH, level=Level._2M_ABOVE_GROUND),
    HRRRVariable(name=VariableName.UGRD, level=Level._10M_ABOVE_GROUND),
    HRRRVariable(name=VariableName.VGRD, level=Level._10M_ABOVE_GROUND),
]
start=datetime(2024, 1, 1, 0, 0)
end=datetime(2024, 1, 2, 0, 0)
end=datetime.now()
requests = create_requests(
    locations=locations,
    variables=vars,
    start=start,
    end=end
)
print(f"created {len(requests)} requests")

created 2016 requests


In [2]:
from pyhrrrzarr.hrrrzarr.fetch import get_all_request_values

await get_all_request_values(requests)


Fetching HRRR data:   0%|          | 0/2016 [00:00<?, ?it/s]

In [3]:
from pyhrrrzarr.hrrrzarr.postprocess import (
    requests_to_df, add_wind_speed_and_direction
)

df = requests_to_df([r for r in requests if r.value is not None])
df = add_wind_speed_and_direction(df)
df.tail()

,location,lat,lon,run_hour,var_name,var_level,value,units
2683,OJNR,43.025,-89.54,2024-01-01 23:00:00,WIND_DIR,10m_above_ground,236.218599,degrees
2684,RNLD,45.655,-89.27,2024-01-01 23:00:00,WIND_DIR,10m_above_ground,236.873897,degrees
2685,SODO,44.779,-87.36,2024-01-01 23:00:00,WIND_DIR,10m_above_ground,243.113293,degrees
2686,SPNR,45.827,-91.86,2024-01-01 23:00:00,WIND_DIR,10m_above_ground,207.520608,degrees
2687,WCRS,44.185,-90.74,2024-01-01 23:00:00,WIND_DIR,10m_above_ground,228.188670,degrees


In [4]:
df.to_parquet("wisco.parquet")
df.to_csv("wisco.csv")